In [1]:
# Import packages
from glob import glob
import pandas as pd
import numpy as np
import csv


In [2]:
# Read files into dataframe
files = glob('data/LoanStats*.csv')
df_list = []

for f in files:
    df_list.append(pd.read_csv(f, header=1, dtype=object))

loan_df = pd.concat(df_list)


In [3]:
# Keep columns with less than 30% null values
keep_cols = []

for col in loan_df.columns:
    if loan_df[col].isna().sum() < len(loan_df.index) * 0.3:
        keep_cols.append(col)     

loan_df = loan_df[keep_cols]

# Drop rows with no loan amount
loan_df = loan_df[loan_df.loan_amnt.notnull()]

# Reset dataframe index
loan_df = loan_df.reset_index(drop=True)


In [4]:
# Read column data types legend from CSV file into list
with open('data/col_data_types.csv', 'r') as file:
    reader = csv.reader(file)
    dtypes_list = list(reader)

# Create list of columns for each data type
floats = []
percentages = []
strings = []
categories = []
dates = []

for i in range(len(dtypes_list)):
    if dtypes_list[i][1] == 'float':
        floats.append(dtypes_list[i][0])
    elif dtypes_list[i][1] == 'percentage':
        percentages.append(dtypes_list[i][0])
    elif dtypes_list[i][1] == 'str':
        strings.append(dtypes_list[i][0])
    elif dtypes_list[i][1] == 'category':
        categories.append(dtypes_list[i][0])
    elif dtypes_list[i][1] == 'date':
        dates.append(dtypes_list[i][0])

# Set column data types
for col in loan_df.columns:
    if col in floats:
        loan_df[col] = loan_df[col].astype(float)
    elif col in percentages:
        loan_df[col] = loan_df[col].astype(str).str.strip('%').astype(float) / 100
    elif col in strings:
        loan_df[col] = loan_df[col].astype(str)
    elif col in categories:
        loan_df[col] = loan_df[col].astype('category')
    elif col in dates:
        loan_df[col] = pd.to_datetime(loan_df[col])
               

In [5]:
# Define general loan status groups
loan_df['status_grp'] = loan_df['loan_status'].map({'Fully Paid':'Good', 'Current':'Good', 'In Grace Period':'Good', 
                                                    'Does not meet the credit policy. Status:Fully Paid':'Good', 
                                                    'Charged Off':'Bad', 'Late (16-30 days)':'Bad', 
                                                    'Late (31-120 days)':'Bad', 'Default':'Bad', 
                                                    'Does not meet the credit policy. Status:Charged Off':'Bad'})

# Define general employment length groups
loan_df['emp_length_grp'] = loan_df['emp_length'].map({'< 1 year':'< 5 years', '1 year':'< 5 years', '2 years':'< 5 years',
                                                      '3 years':'< 5 years', '4 years':'< 5 years', '5 years':'5-9 years',
                                                      '6 years':'5-9 years', '7 years':'5-9 years', '8 years':'5-9 years',
                                                      '9 years':'5-9 years', '10+ years':'10+ years'})

# Define home ownership groups
loan_df['home_ownership_grp'] = loan_df['home_ownership'].map({'MORTGAGE':'MORTGAGE', 'RENT':'RENT', 'OWN':'OWN',
                                                              'ANY':'OTHER', 'OTHER':'OTHER', 'NONE':'OTHER'})


In [6]:
# Read income by zip code data into dataframe
census_file = 'data/US_IncomeByZip.xlsx'
census_df = pd.read_excel(census_file, dtype=object)

# Filter and rename columns in dataframe
census_df = census_df[['Zip (ZCTA)', 'Median household income in the past 12 months (in 2017 inflation-adjusted dollars)', 
                      'Household Income in the past 12 months (in 2017 inflation adjusted dollars)']]
census_df.columns = ['zip_code_full', 'median_hh_income', 'households']

# Set column data types
census_df['zip_code_full'] = census_df['zip_code_full'].astype(str)
census_df['median_hh_income'] = pd.to_numeric(census_df['median_hh_income'], errors='coerce')
census_df['households'] = pd.to_numeric(census_df['households'], errors='coerce')

# Drop rows with null values
census_df = census_df[census_df.zip_code_full != 'nan']
census_df = census_df[census_df.median_hh_income != -1]


In [11]:
# Aggregate by first 3 digits of zip code and calculate weighted average
census_df['zip_code'] = census_df['zip_code_full'].str[:3] + 'xx'

def weighted_average(df, data_col, weight_col, by_col):
    df['data_x_weight'] = df[data_col] * df[weight_col]
    df['weight_where_notnull'] = df[weight_col] * pd.notnull(df[data_col])
    g = df.groupby(by_col)
    result = g['data_x_weight'].sum() / g['weight_where_notnull'].sum()
    del df['data_x_weight'], df['weight_where_notnull']
    return result

# Create dataframe of weighted average median income by zip code group
zip_code_df = pd.DataFrame(weighted_average(census_df, 'median_hh_income', 'households', 'zip_code'))
zip_code_df.columns = ['zip_code_median_income']
zip_code_df = zip_code_df.reset_index()


In [12]:
# Merge loans dataframe with income-by-zip-code dataframe
df = pd.merge(loan_df, zip_code_df, how='left')

# Calculate ratio of annual income to zip code's median income
df['income_vs_peers'] = df['annual_inc'] / df['zip_code_median_income']


In [13]:
# Export clean dataframe to CSV file
df.to_csv('clean_df.csv')
